In [1]:
import sys
import importlib
sys.path.append(r'../../pykneer')
sys.path.append(r'../pykneer')
import pykneer_io as io
importlib.reload(io)
import preprocessing_for_nb as prep
importlib.reload(prep)
import segmentation_sa_for_nb as segm
importlib.reload(segm)

import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append("../../..")
from MyProcessDicom import *

from preparing_images import prepare_image_and_list, prepare_centerline
from display_and_show_images import *

In [2]:
def load_data(data_type):   ## just for convinience
    if data_type == "newsubject":
        input_file_name = "./image_list_newsubject.txt"
        modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
    elif data_type == "reference":
        input_file_name = "./image_list_reference.txt"
        modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal"
    elif data_type == "longitudinal":
        input_file_name = "./image_list_longitudinal.txt"
        modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
    
    n_of_cores      = 1
    image_data = io.load_image_data_segmentation(modality, input_file_name)
    return image_data

In [17]:
# process list
# first in TP as reference
caselist = [
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9189303','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9939718','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9939718','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9941231','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9941231','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9942724','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9942724','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9941446','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9941446','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9404954','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9404954','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
    {'pid': '9942030','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'L'},
    {'pid': '9942030','TP':[0,1,2,3,4,5,6,8,10],'TP2':[0,12,18,24,30,36,48,72,96],'side':'R'},
]

prepare_image_and_list(caselist)
prepare_centerline(caselist)

ahaknee18tp29  exist
ahaknee24tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee48tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee30tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee72tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee36tp29  exist
ahaknee18tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist
ahaknee18tp29  exist
ahaknee30tp29  exist
ahaknee72tp29  exist
ahaknee96tp29  exist


In [4]:
def preprocess():
    input_file_name           = ".\image_list_preprocessing.txt"
    n_of_cores                = 1 # change the number of cores according to your computer
    intensity_standardization = 0

    # Image information
    image_data = io.load_image_data_preprocessing(input_file_name)

    ref = sitk.ReadImage('../../reference/newsubject/reference.mha')

    # Spatial standardization
    # Step 0: Read the dicom images
    prep.read_dicom_stack(image_data, n_of_cores)
    # Save image header
    prep.print_dicom_header(image_data, n_of_cores)    # save to *.txt

    # Step 1: Change orientation to RAI
    #prep.axial_to_sagittal(image_data, n_of_cores)
    prep.orientation_to_rai(image_data, n_of_cores)

    # Step 2: Change laterality if knee is right
    prep.flip_rl(image_data, n_of_cores)
    # Step 3: Set image origin to (0,0,0)
    prep.origin_to_zero(image_data, n_of_cores)    # Images are saved as *_orig.mha and they are anonymized

    # 4. Correct magnetic field inhomogeneities
    # Magnetic fields inhomogeneities create grey shades on images. This correction removes these shades. 
    # This is the longest step of the processing. It can take up to 15-20 min on a standard PC or laptop
    if intensity_standardization == 1:
        prep.field_correction(image_data, n_of_cores)
        prep.rescale_to_range(image_data, n_of_cores)  # 5. Rescale intensities to [0 100]
        prep.edge_preserving_smoothing(image_data, n_of_cores)  # 6. Edge preserving smoothing

# preprocess()

In [5]:
def segmentation_sa():
    input_file_name = "./image_list_newsubject.txt"
    modality        = "newsubject" # use "newsubject", "longitudinal", or "multimodal" 
    n_of_cores      = 1

    image_data = io.load_image_data_segmentation(modality, input_file_name)
    segm.prepare_reference(image_data)
    ### NOTE!!! reference_fv is dummy!!!

    # Segment Bone
#     # 1. Register image to reference
    segm.register_bone_to_reference(image_data, n_of_cores)
#     # 2. Invert transformations
    segm.invert_bone_transformations(image_data, n_of_cores)
#     # 3. Warp reference mask to moving image
    segm.warp_bone_mask(image_data, n_of_cores)
    
    segm.register_tibia_to_reference(image_data, n_of_cores)
    segm.invert_tibia_transformations(image_data, n_of_cores)
    segm.warp_tibia_mask(image_data, n_of_cores)

    # Segment cartilage
    segm.register_cartilage_to_reference(image_data, n_of_cores)
    segm.invert_cartilage_transformations(image_data, n_of_cores)
    segm.warp_cartilage_mask(image_data, n_of_cores)
    
# segmentation_sa()

-> image information loaded
reference.mha
-> Reference preparation completed
-> Registration completed
-> The total time was 90.98 seconds (about 1 min)
-> Inversion completed
-> The total time was 99.82 seconds (about 1 min)
-> Warping completed
-> The total time was 24.40 seconds (about 0 min)


In [27]:
def segmentation_sa_long():
    input_file_name = "./image_list_longitudinal.txt"
    modality        = "longitudinal" # use "newsubject", "longitudinal", or "multimodal" 
    n_of_cores      = 1

    image_data = io.load_image_data_segmentation(modality, input_file_name)
#     segm.prepare_reference(image_data)

    segm.register_bone_to_reference(image_data, n_of_cores)
    ## optioned
    segm.invert_bone_transformations(image_data, n_of_cores)
    segm.warp_tibia_mask(image_data, n_of_cores)
    segm.warp_bone_mask(image_data, n_of_cores)
    
    segm.warp_centerline(image_data, n_of_cores)

    # segm.register_vessel_to_reference(image_data, n_of_cores)
    # segm.invert_vessel_transformations(image_data, n_of_cores)
    # segm.warp_vessel_mask(image_data, n_of_cores)
    
#     segm.register_cartilage_to_reference(image_data, n_of_cores)
#     segm.invert_cartilage_transformations(image_data, n_of_cores)
#     segm.warp_cartilage_mask(image_data, n_of_cores)
    
# segmentation_sa_long()

-> image information loaded
-> Warping completed
-> The total time was 52.85 seconds (about 0 min)


In [6]:
preprocess()
segmentation_sa()
segmentation_sa_long()

In [15]:
# # mask_file_name = image_data[0]['reference_folder'] + '9941231L_TP0_prep_f.mha'
# mask_file_name = image_data[0]['segmented_folder'] + '9404954L_TP0_prep_f.mha'
# # mask_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\newsubject\\reference_f.mha"
# tibia_file_name = image_data[0]['reference_folder'] + '9404954L_TP0_prep_t.mha'
# # tibia_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9404954L_TP3_prep_t.mha"
# # mask_file_name = 'C:\\Zhixuan\\OAI-registration\\pykneer-yg\\reference\\longitudinal\\9941446L_TP0_prep_t.mha'
# moving_file_name   = image_data[0]['moving_folder']    + "9404954L_TP0_prep.mha"
# # moving_file_name = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\registered\\9404954L_TP1_prep\\t_rigid.mha"
# # tmp_file_name  = "C:\\Zhixuan\\OAI-registration\\pykneer-yg\\registered\\9404954L_TP0_prep\\f_rigid.mha"
# mask = sitk.ReadImage(mask_file_name)
# tibia = sitk.ReadImage(tibia_file_name)
# moving = sitk.ReadImage(moving_file_name)
# key = 1
# if key == 1:
#     mask = resample_bwimage(mask)
#     tibia = resample_bwimage(tibia)
#     moving = resample_bwimage(moving)
# #     show_masked_images_sagittal(moving, mask)
#     show_all_masked_images(moving, mask,tibia, 0)
# else:
# #     show_all_masked_images(moving, mask,tibia)
#     show_masked_images(moving, tibia)    

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [22]:
## TP0 (baseline result)
image_data = load_data("reference")
show_sagital_reference(image_data, key = 1)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [23]:
image_data = load_data("longitudinal")
# rigid_only
show_sagital_slice_images(image_data, is_rigid = True)
# show_sagital_slice_images(image_data) # non-rigid (add spline)

9189303L_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP1


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9189303R_TP3


...

interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP5


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

9942030R_TP6


interactive(children=(IntSlider(value=191, description='image_z', max=383), FloatSlider(value=0.5, description…

In [ ]:
# 9939718L_TP10, 9939718R_TP10, 9941231L_TP10, 9942724L_TP10, 9942724R_TP10,
# 9941446R_TP1, 9404954L_TP3, 9404954L_TP10

In [ ]:
# transverse view    key = 0(mask+vessel) key = 1(mask)  key=2(vessel)
show_registered_result(image_data, key=0)

In [4]:
# reference-moving blend result
show_slice_images(image_data)

9189303L_TP1


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9189303R_TP1


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9189303R_TP3


...

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9942030R_TP5


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

9942030R_TP6


interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [ ]:
#######################################################################
####  evaluation part
#######################################################################

In [ ]:
sys.path.append("../../..")
from compute_metrics import *
def compute_evaluation():
    image_data = load_data("longitudinal")
    quality = compute_quality_metrics(image_data)
    compute_centerline_metrics(image_data)
compute_evaluation()

In [29]:
# compute_centerline_metrics(image_data)

-> image information loaded
9189303L_TP1:	0.5872910199598248 74
9189303R_TP1:	1.0166063738280133 75
9189303R_TP3:	1.7020588645966697 74
9189303R_TP5:	1.7362594598737633 75
9189303R_TP6:	2.2539160671057874 75
9189303R_TP8:	2.5524522009339194 75
9939718L_TP1:	0.9694847681549441 60
9939718L_TP3:	0.8705975611035319 60
9939718L_TP5:	1.6549732839559583 60
9939718L_TP6:	1.6740662713262966 60
9939718L_TP8:	3.0409945356650736 60
9939718R_TP1:	1.120859785434382 60
9939718R_TP2:	2.325657436678292 60
9939718R_TP3:	1.4239781021419153 60
9939718R_TP5:	0.9046755107013374 59
9939718R_TP6:	1.6728009288067134 60
9939718R_TP8:	2.2424092841305208 59
9939718R_TP10:	2.495634770439222 55
9941231L_TP1:	1.6243590894328959 68
9941231L_TP3:	1.0539818994481445 69
9941231L_TP5:	4.076624508773793 69
9941231L_TP6:	1.2942439341862528 69
9941231L_TP8:	1.9164967146912035 69
9941231R_TP1:	2.932053382474676 74
9941231R_TP3:	2.6642168766461714 73
9941231R_TP5:	3.2682496775421295 73
9941231R_TP6:	3.1901675076959295 74
9942